In [ ]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig, EncoderDecoderModel, BertConfig, EncoderDecoderConfig, BertLMHeadModel, BertTokenizer
from peft import get_peft_model

In [ ]:
model_name = "gemma-2b"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(model)

In [ ]:
input_text = "What should I do on a trip to Europe?"
input_ids = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**input_ids, max_length=128)
print(tokenizer.decode(outputs[0]))

In [ ]:
input_text = "Explain the process of photosynthesis in a way taht a child could understand"
input_ids = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**input_ids, max_length=128)
print(tokenizer.decode(outputs[0]))

In [ ]:
from datasets import load_dataset

dataset_name = "databricks/databricks-dolly-15k"
dataset = load_dataset(dataset_name, split="train[0:1000]")

print(f"Instruction is: {dataset[0]['instruction']}")
print(f"Response is: {dataset[0]['response']}")

In [ ]:
from trl import SFTConfig, SFTTrainer

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        if example['category'][i] in ['open_qa', 'general_qa']:
            text = f"Instruction:\n{example['instruction']}\n\nResponse:\n{example['response']}"
            output_texts.append(text)
    return output_texts

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    formatting_func=formatting_prompts_func
)

In [ ]:
trainer.train()